In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/wiki_movie_plots_deduped.csv


In [2]:
import re
import numpy as np
import pandas as pd

# Gensim 
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

#nltk - stopwords
from nltk.corpus import stopwords

In [4]:
stop_words = stopwords.words('english')
# stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [5]:
#Import the dataset
data=pd.read_csv('/kaggle/input/wiki_movie_plots_deduped.csv')
data.head()

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr..."
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov..."
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed..."
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...


In [7]:
# Some cleaning with regex
# function for cleaning the plots of the movies
def clean_text(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "can not ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
 
    text = text.strip(' ')
    return text

In [8]:
data['CleanedPlot'] = data['Plot'].apply(clean_text)

In [11]:
data['CleanedPlot'][100]

'grace roberts (played by lea leland), marries rancher edward smith, who is revealed to be a neglectful, vice-ridden spouse. they have a daughter, vivian. dr. franklin (leonid samoloff) whisks grace away from this unhappy life, and they move to new york under aliases, pretending to be married (since surely smith would not agree to a divorce). grace and franklin have a son, walter (milton s. gould). vivian gets sick, however, and grace and franklin return to save her. somehow this reunion, as smith had assumed grace to be dead, causes the death of franklin. this plot device frees grace to return to her father  farm with both children.[1]'

In [13]:
# tokenize each sentence into a list of words, 
#removing punctuations and unnecessary characters altogether
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  
        # deacc=True removes punctuations

data_words = list(sent_to_words(data['CleanedPlot']))

print(data_words[:1])


[['bartender', 'is', 'working', 'at', 'saloon', 'serving', 'drinks', 'to', 'customers', 'after', 'he', 'fills', 'stereotypically', 'irish', 'man', 'bucket', 'with', 'beer', 'carrie', 'nation', 'and', 'her', 'followers', 'burst', 'inside', 'they', 'assault', 'the', 'irish', 'man', 'pulling', 'his', 'hat', 'over', 'his', 'eyes', 'and', 'then', 'dumping', 'the', 'beer', 'over', 'his', 'head', 'the', 'group', 'then', 'begin', 'wrecking', 'the', 'bar', 'smashing', 'the', 'fixtures', 'mirrors', 'and', 'breaking', 'the', 'cash', 'register', 'the', 'bartender', 'then', 'sprays', 'seltzer', 'water', 'in', 'nation', 'face', 'before', 'group', 'of', 'policemen', 'appear', 'and', 'order', 'everybody', 'to', 'leave']]


In [14]:
#Creating Bigram and Trigram Models
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['bartender', 'is', 'working', 'at', 'saloon', 'serving', 'drinks', 'to', 'customers', 'after', 'he', 'fills', 'stereotypically', 'irish', 'man', 'bucket', 'with', 'beer', 'carrie', 'nation', 'and', 'her', 'followers', 'burst', 'inside', 'they', 'assault', 'the', 'irish', 'man', 'pulling', 'his', 'hat', 'over', 'his', 'eyes', 'and', 'then', 'dumping', 'the', 'beer', 'over', 'his', 'head', 'the', 'group', 'then', 'begin', 'wrecking', 'the', 'bar', 'smashing', 'the', 'fixtures', 'mirrors', 'and', 'breaking', 'the', 'cash_register', 'the', 'bartender', 'then', 'sprays', 'seltzer', 'water', 'in', 'nation', 'face', 'before', 'group', 'of', 'policemen', 'appear', 'and', 'order', 'everybody', 'to', 'leave']


In [15]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [17]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['bartender', 'work', 'saloon', 'serve', 'drink', 'customer', 'fill', 'stereotypically', 'irish', 'man', 'bucket', 'beer', 'carrie', 'nation', 'follower', 'burst', 'assault', 'irish', 'man', 'pull', 'hat', 'eye', 'dump', 'beer', 'head', 'group', 'begin', 'wreck', 'bar', 'smash', 'fixture', 'mirror', 'break', 'cash_register', 'bartender', 'spray', 'seltzer', 'water', 'nation', 'face', 'group', 'policeman', 'appear', 'order', 'everybody', 'leave']]


In [18]:
# Create the Dictionary and Corpus needed for Topic Modeling
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 2), (4, 2), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 2), (21, 1), (22, 1), (23, 2), (24, 1), (25, 2), (26, 1), (27, 2), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1)]]


In [19]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('appear', 1),
  ('assault', 1),
  ('bar', 1),
  ('bartender', 2),
  ('beer', 2),
  ('begin', 1),
  ('break', 1),
  ('bucket', 1),
  ('burst', 1),
  ('carrie', 1),
  ('cash_register', 1),
  ('customer', 1),
  ('drink', 1),
  ('dump', 1),
  ('everybody', 1),
  ('eye', 1),
  ('face', 1),
  ('fill', 1),
  ('fixture', 1),
  ('follower', 1),
  ('group', 2),
  ('hat', 1),
  ('head', 1),
  ('irish', 2),
  ('leave', 1),
  ('man', 2),
  ('mirror', 1),
  ('nation', 2),
  ('order', 1),
  ('policeman', 1),
  ('pull', 1),
  ('saloon', 1),
  ('seltzer', 1),
  ('serve', 1),
  ('smash', 1),
  ('spray', 1),
  ('stereotypically', 1),
  ('water', 1),
  ('work', 1),
  ('wreck', 1)]]

In [26]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=14,
                                           alpha='auto',
                                           per_word_topics=True)

In [28]:
# Print the Keyword in the 10 topics
from pprint import pprint
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.204*"father" + 0.137*"mother" + 0.129*"son" + 0.052*"parent" + '
  '0.033*"college" + 0.026*"truth" + 0.021*"always" + 0.021*"class" + '
  '0.016*"top" + 0.015*"identity"'),
 (1,
  '0.216*"red" + 0.059*"fun" + 0.018*"satisfy" + 0.011*"pierre" + '
  '0.009*"battery" + 0.006*"tender" + 0.006*"january" + 0.006*"valentine" + '
  '0.006*"railroad" + 0.006*"overboard"'),
 (2,
  '0.022*"get" + 0.022*"take" + 0.020*"go" + 0.016*"find" + 0.016*"friend" + '
  '0.016*"leave" + 0.015*"tell" + 0.015*"come" + 0.015*"make" + 0.014*"day"'),
 (3,
  '0.030*"money" + 0.025*"night" + 0.024*"plan" + 0.017*"work" + 0.016*"town" '
  '+ 0.015*"member" + 0.015*"show" + 0.014*"steal" + 0.013*"former" + '
  '0.012*"company"'),
 (4,
  '0.135*"fake" + 0.055*"forever" + 0.045*"laboratory" + 0.037*"mentor" + '
  '0.028*"bright" + 0.020*"ward" + 0.018*"lawrence" + 0.017*"deject" + '
  '0.015*"greek" + 0.013*"melodrama"'),
 (5,
  '0.037*"village" + 0.028*"force" + 0.027*"fight" + 0.027*"order" + '
  '0.024*"

In [29]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -18.145340863854567

Coherence Score:  0.33465060703644334


In [30]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.320934 -0.166384       1        1  43.215103
10    -0.292819 -0.179606       2        1  12.536168
3     -0.245897 -0.049111       3        1  10.375087
6     -0.211081  0.215467       4        1   8.297790
5     -0.272574 -0.129519       5        1   7.859304
16    -0.160771  0.095108       6        1   4.400213
0     -0.086607  0.264923       7        1   3.365646
9     -0.098735  0.185719       8        1   3.208017
17     0.049559  0.033231       9        1   1.294937
13     0.048985  0.033182      10        1   1.030792
11     0.130454 -0.022903      11        1   0.729741
14     0.115981 -0.013493      12        1   0.715798
19     0.098686 -0.014413      13        1   0.658781
15     0.116547 -0.012975      14        1   0.622111
7      0.149765 -0.030407      15        1   0.478610
12     0.131312 -0.022369      16        1   0.430016
4      0.203711 -0.045491      17        1   0.251807
8      0.199178 -0.044680      18        1   0.225907
18     0.229479 -0.048995      19        1   0.157042
1      0.215761 -0.047282      20        1   0.147133, topic_info=       Category          Freq          Term         Total  loglift  logprob
term                                                                       
178     Default  64502.000000          kill  64502.000000  30.0000  30.0000
576     Default  46293.000000        father  46293.000000  29.0000  29.0000
860     Default  49588.000000          love  49588.000000  28.0000  28.0000
182     Default  31023.000000        mother  31023.000000  27.0000  27.0000
190     Default  29295.000000           son  29295.000000  26.0000  26.0000
82      Default  27825.000000          film  27825.000000  25.0000  25.0000
336     Default  37125.000000        family  37125.000000  24.0000  24.0000
446     Default  29561.000000        police  29561.000000  23.0000  23.0000
72      Default  33076.000000         young  33076.000000  22.0000  22.0000
25      Default  46589.000000           man  46589.000000  21.0000  21.0000
340     Default  37617.000000          life  37617.000000  20.0000  20.0000
282     Default  25942.000000      daughter  25942.000000  19.0000  19.0000
275     Default  14741.000000           car  14741.000000  18.0000  18.0000
853     Default  21963.000000         story  21963.000000  17.0000  17.0000
170     Default  64672.000000          find  64672.000000  16.0000  16.0000
345     Default  21642.000000          wife  21642.000000  15.0000  15.0000
51      Default  64454.000000           get  64454.000000  14.0000  14.0000
71      Default  21099.000000         woman  21099.000000  13.0000  13.0000
654     Default   9468.000000           boy   9468.000000  12.0000  12.0000
1270    Default  23111.000000         death  23111.000000  11.0000  11.0000
1234    Default  10197.000000          team  10197.000000  10.0000  10.0000
589     Default  19624.000000       village  19624.000000   9.0000   9.0000
337     Default  59571.000000            go  59571.000000   8.0000   8.0000
898     Default  25575.000000          live  25575.000000   7.0000   7.0000
509     Default  24168.000000         fight  24168.000000   6.0000   6.0000
1575    Default  21149.000000         money  21149.000000   5.0000   5.0000
330     Default   9592.000000         train   9592.000000   4.0000   4.0000
776     Default  18667.000000         marry  18667.000000   3.0000   3.0000
719     Default  24350.000000          year  24350.000000   2.0000   2.0000
760     Default  14570.000000        murder  14570.000000   1.0000   1.0000
...         ...           ...           ...           ...      ...      ...
438     Topic20     56.399548     overboard     58.068321   6.4924  -5.1674
2356    Topic20     33.818195        harris     35.486969   6.4734  -5.6789
499     Topic20     25.646076          weed     27.314852   6.4585  -5.9555
2368    Topic20      0.497474    